Imports & device & path

In [1]:
import sys
from pathlib import Path

# Get project root from current working directory
PROJECT_ROOT = Path.cwd().parent  # parent = يرجع خطوة لفوق (من notebooks → root)
SRC_DIR = PROJECT_ROOT / "src"

# Add src/ to Python path
sys.path.append(str(SRC_DIR))

print("SRC_DIR added:", SRC_DIR)


# Cell 1
import os, sys
from pathlib import Path
import torch
import torch.nn as nn

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
SRC_DIR = PROJECT_ROOT / "src"
sys.path.append(str(SRC_DIR))

from attention import causal_mask, MultiHeadSelfAttention, TransformerBlock

device = "cuda" if torch.cuda.is_available() else "cpu"
device


SRC_DIR added: c:\Users\ASUS\OneDrive\Desktop\llm-from-scratch-tokenization\src


'cpu'

In [2]:
# Cell 2
T = 12
mask = causal_mask(T)
print(mask.shape)
print(mask[:6, :6].int())  


torch.Size([12, 12])
tensor([[0, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0]], dtype=torch.int32)


In [3]:
# Cell 3
torch.manual_seed(42)
B, T, C, H = 2, 16, 64, 8

x = torch.randn(B, T, C).to(device)
mha = MultiHeadSelfAttention(d_model=C, n_heads=H, dropout=0.1).to(device)
y = mha(x)
y.shape


torch.Size([2, 16, 64])

In [4]:
# Cell 4
block = TransformerBlock(d_model=C, n_heads=H, mlp_ratio=4, dropout=0.1).to(device)
out = block(x)
out.shape


torch.Size([2, 16, 64])

In [5]:
# Cell 5
import tiktoken
enc = tiktoken.get_encoding("gpt2")
vocab_size = enc.n_vocab
d_model = 64
context_len = 64

tok_emb = nn.Embedding(vocab_size, d_model).to(device)
pos_emb = nn.Embedding(context_len, d_model).to(device)

text = "Self-attention lets tokens attend only to previous positions in decoder-only models."
ids = enc.encode(text)
ids = ids[:context_len] 
T = len(ids)

x_ids = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)  # (1,T)

tokens = tok_emb(x_ids)  # (1,T,C)
positions = pos_emb(torch.arange(T, device=device)).unsqueeze(0)  # (1,T,C)
inp = tokens + positions  # (1,T,C)

mha_small = MultiHeadSelfAttention(d_model=d_model, n_heads=4, dropout=0.0).to(device)
with torch.no_grad():
    out = mha_small(inp)
out.shape


torch.Size([1, 18, 64])

In [6]:
# Cell 6 
import time
torch.manual_seed(0)
B, T, C, H = 4, 256, 512, 8
x = torch.randn(B, T, C, device=device)
mha = MultiHeadSelfAttention(d_model=C, n_heads=H, dropout=0.0).to(device)

# warmup
for _ in range(5):
    _ = mha(x)

start = time.time()
for _ in range(20):
    _ = mha(x)
torch.cuda.synchronize() if device == "cuda" else None
elapsed = time.time() - start
print(f"20 iters took {elapsed:.3f}s on {device}")


20 iters took 0.173s on cpu
